In [1]:
# ATSIT 2019: dimensional speech emotion recognition from text feature

# uncomment these to run on CPU only
import os
#os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
#os.environ["CUDA_VISIBLE_DEVICES"]=""

import numpy as np
import matplotlib.pyplot as plt
import pickle
import keras.backend as K
import pandas as pd
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Masking,BatchNormalization, LSTM, TimeDistributed, Bidirectional, Embedding, Dropout, Flatten, concatenate, Convolution1D
from keras.utils import to_categorical
from sklearn.preprocessing import MinMaxScaler

from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from pytorch_pretrained_bert.modeling import BertModel
from pytorch_pretrained_bert import BertTokenizer
from keras.callbacks import EarlyStopping
import codecs

import io
import random as rn
import tensorflow as tf

Using TensorFlow backend.


In [2]:
rn.seed(123)
np.random.seed(99)
tf.set_random_seed(1234)


# Concordance correlation coefficient (CCC)-based loss function - using non-inductive statistics
def ccc(gold, pred):
    gold       = K.squeeze(gold, axis=-1)
    pred       = K.squeeze(pred, axis=-1)
    gold_mean  = K.mean(gold, axis=-1, keepdims=True)
    pred_mean  = K.mean(pred, axis=-1, keepdims=True)
    covariance = (gold-gold_mean)*(pred-pred_mean)
    gold_var   = K.mean(K.square(gold-gold_mean), axis=-1,  keepdims=True)
    pred_var   = K.mean(K.square(pred-pred_mean), axis=-1, keepdims=True)
    ccc        = K.constant(2.) * covariance / (gold_var + pred_var + K.square(gold_mean - pred_mean) + K.common.epsilon())
    return ccc


def ccc_loss(gold, pred):  
    # input (num_batches, seq_len, 1)
    ccc_loss   = K.constant(1.) - ccc(gold, pred)
    return ccc_loss


In [3]:
# other parameters
#MAX_SEQUENCE_LENGTH = 554
EMBEDDING_DIM = 768 #1024
nb_words = 3438
#path = '/home/s1820002/IEMOCAP-Emotion-Detection/'
feat = np.load('../../dimensionalSM/data/feat_34_hfs.npy')
data = np.load('../../dimensionalSM/IEMOCAP_full_release/data_collected_10039.pickle', allow_pickle=True)
text = [t['transcription'] for t in data]
vad = np.load('../../dimensionalSM/data/y_egemaps.npy')

# remove outlier, < 1, > 5
vad = np.where(vad==5.5, 5.0, vad)
vad = np.where(vad==0.5, 1.0, vad)

scaled_vad = True

# standardization
if scaled_vad:
    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaler = scaler.fit(vad) #.reshape(vad.shape[0]*vad.shape[1], vad.shape[2]))
    scaled_vad = scaler.transform(vad) #.reshape(vad.shape[0]*vad.shape[1], vad.shape[2]))
    vad = scaled_vad 
else:
    vad = vad

# load fastext model
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text)

token_tr_X = tokenizer.texts_to_sequences(text)
MAX_SEQUENCE_LENGTH = len(max(token_tr_X, key=len))
x_train_text = []
x_train_text = sequence.pad_sequences(token_tr_X, maxlen=MAX_SEQUENCE_LENGTH)

# word_index = tokenizer.word_index
# nb_words = len(word_index) +1
# print('Found %s unique tokens' % len(word_index))

def get_bert_embed_matrix():
    bert = BertModel.from_pretrained(BERT_FP)
    bert_embeddings = list(bert.children())[0]
    bert_word_embeddings = list(bert_embeddings.children())[0]
    mat = bert_word_embeddings.weight.data.numpy()
    return mat


def get_bert_embed_matrix():
    bert = BertModel.from_pretrained('bert-base-uncased')
    bert_embeddings = list(bert.children())[0]
    bert_word_embeddings = list(bert_embeddings.children())[0]
    mat = bert_word_embeddings.weight.data.numpy()
    return mat


g_word_embedding_matrix = get_bert_embed_matrix()

# # load model
# EMBEDDING_DIM = 300
# file_loc = '../../data/glove.42B.300d.txt'
# print (file_loc)

# gembeddings_index = {}
# with codecs.open(file_loc, encoding='utf-8') as f:
#     for line in f:
#         values = line.split(' ')
#         word = values[0]
#         gembedding = np.asarray(values[1:], dtype='float32')
#         gembeddings_index[word] = gembedding
# #
# f.close()
# print('G Word embeddings:', len(gembeddings_index))

# nb_words = len(word_index) +1
# g_word_embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))
# for word, i in word_index.items():
#     gembedding_vector = gembeddings_index.get(word)
#     if gembedding_vector is not None:
#         g_word_embedding_matrix[i] = gembedding_vector
        
# print('G Null word embeddings: %d' % np.sum(np.sum(g_word_embedding_matrix, axis=1) == 0))

# split train/test
# split = 7869
# earlystop = EarlyStopping(monitor='val_loss', patience=10, mode='min', 
#                           restore_best_weights=True)

# # remove outlier
# outlier = np.array([1674, 3427, 5086, 5093, 5096, 5104, 7821])
# mask = np.ones(len(vad), np.bool)
# mask[outlier] = 0
# vad = vad[mask]
# x_train_text = x_train_text[mask]
# model: GRU


100%|██████████| 407873900/407873900 [01:11<00:00, 5722058.46B/s] 


In [4]:
#split train/test
split = 7869

In [5]:
# API model
def model():
    # speech network
    input_speech = Input(shape=(feat.shape[1], feat.shape[2]), name='speech_input')
    net_speech = BatchNormalization()(input_speech)
    net_speech = LSTM(32, return_sequences=True)(net_speech)
    net_speech = LSTM(64, return_sequences=True)(net_speech)
    net_speech = LSTM(128, return_sequences=True)(net_speech)
    net_speech = LSTM(64, return_sequences=True)(net_speech)
    net_speech = LSTM(32, return_sequences=True)(net_speech)
    model_speech = Flatten()(net_speech)
    #model_speech = Dropout(0.1)(net_speech)
    
    #text network
    input_text = Input(shape=(MAX_SEQUENCE_LENGTH, ))
    net_text = Embedding(*g_word_embedding_matrix.shape,
                    weights = [g_word_embedding_matrix],
                    trainable = True)(input_text)
    net = Bidirectional(LSTM(128, return_sequences=True))(net_text)
    net = Bidirectional(LSTM(64, return_sequences=False))(net)  
    model_text = Dense(64)(net)

    # combined model
    model_combined = concatenate([model_speech, model_text])
    model_combined = Dense(64, activation='relu')(model_combined)
    model_combined = Dense(32, activation='relu')(model_combined)
    #model_combined = Dropout(0.4)(model_combined)
    target_names = ('v', 'a', 'd')
    model_combined = [Dense(1, name=name)(model_combined) for name in target_names]

    model = Model([input_speech, input_text], model_combined) 
    model.compile(loss=ccc_loss,
                  loss_weights={'v': 1, 'a': 1, 'd': 1},
                  optimizer='rmsprop', metrics=[ccc])
    return model


# model_1 = model()
# model_1.summary()

In [6]:
# main function for for LOSO
def main():
    model_1 = model()
    earlystop = EarlyStopping(monitor='val_loss', mode='min', patience=10,
                              restore_best_weights=True)
    hist = model_1.fit( [feat[:split],x_train_text[:split]],
                      vad[:split].T.tolist(), batch_size=32, #best:8
                      validation_split=0.2, epochs=50, verbose=1, shuffle=True,
                      callbacks=[earlystop])
    metrik = model_1.evaluate( [feat[split:],x_train_text[split:]], vad[split:].T.tolist())
    print(metrik[-3:])
    return metrik
    
if __name__ == '__main__':
    #loss_weight_step = np.around(np.arange(0.0, 1.1, 0.1), decimals=1)
    #dropout_rates = np.arange(0.5, 0.8, 0.1)
    weight = []
    #for i in dropout_rates:
    for time in range(30):
        best = main()
        weight.append([time, best[-3], best[-2], best[-1], np.mean(best[-3:])])
        print(time)
    weight_df = pd.DataFrame(weight)
    weight_df.to_csv('speech_textBERT_multi7869.csv', header=['time', 'v','a','d', 'ave'])

Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Train on 6295 samples, validate on 1574 samples
Epoch 1/50
6295/6295 [==============================] - 137s 22ms/step - loss: 1.6452 - v_loss: 0.6538 - a_loss: 0.4533 - d_loss: 0.5374 - v_ccc: 0.3456 - a_ccc: 0.5465 - d_ccc: 0.4626 - val_loss: 2.0243 - val_v_loss: 0.6591 - val_a_loss: 0.7281 - val_d_loss: 0.6473 - val_v_ccc: 0.3492 - val_a_ccc: 0.2743 - val_d_ccc: 0.3522
Epoch 2/50
6295/6295 [==============================] - 137s 22ms/step - loss: 1.1207 - v_loss: 0.3724 - a_loss: 0.3283 - d_loss: 0.4201 - v_ccc: 0.6277 - a_ccc: 0.6717 - d_ccc: 0.5799 - val_loss: 1.7157 - val_v_loss: 0.6376 - val_a_loss: 0.5426 - val_d_loss: 0.5403 - val_v_ccc: 0.3661 - val_a_ccc: 0.4599 - val_d_ccc: 0.4583
Epoch 3/50
6295/6295 [==============================] - 138s 22ms/step - loss: 1.0002 - v_loss: 0.3106 - a_loss: 0.3053 - d_loss: 0.3838 - v_ccc: 0.6892 - a_ccc: 0.6944 - d_ccc: 0.6161 - val_loss: 1.5320 - val_v_loss

Epoch 4/50
6295/6295 [==============================] - 130s 21ms/step - loss: 0.9512 - v_loss: 0.2837 - a_loss: 0.3025 - d_loss: 0.3652 - v_ccc: 0.7164 - a_ccc: 0.6973 - d_ccc: 0.6351 - val_loss: 1.5719 - val_v_loss: 0.6026 - val_a_loss: 0.4319 - val_d_loss: 0.5488 - val_v_ccc: 0.4009 - val_a_ccc: 0.5721 - val_d_ccc: 0.4551
Epoch 5/50
6295/6295 [==============================] - 132s 21ms/step - loss: 0.8700 - v_loss: 0.2490 - a_loss: 0.2831 - d_loss: 0.3379 - v_ccc: 0.7510 - a_ccc: 0.7169 - d_ccc: 0.6621 - val_loss: 1.5020 - val_v_loss: 0.5682 - val_a_loss: 0.4221 - val_d_loss: 0.5224 - val_v_ccc: 0.4348 - val_a_ccc: 0.5825 - val_d_ccc: 0.4808
Epoch 6/50
6295/6295 [==============================] - 132s 21ms/step - loss: 0.8204 - v_loss: 0.2332 - a_loss: 0.2730 - d_loss: 0.3139 - v_ccc: 0.7667 - a_ccc: 0.7269 - d_ccc: 0.6860 - val_loss: 1.5997 - val_v_loss: 0.6037 - val_a_loss: 0.4665 - val_d_loss: 0.5455 - val_v_ccc: 0.4028 - val_a_ccc: 0.5391 - val_d_ccc: 0.4584
Epoch 7/50
6295/629

6295/6295 [==============================] - 110s 17ms/step - loss: 0.5452 - v_loss: 0.1506 - a_loss: 0.1870 - d_loss: 0.2076 - v_ccc: 0.8494 - a_ccc: 0.8129 - d_ccc: 0.7924 - val_loss: 1.5792 - val_v_loss: 0.5705 - val_a_loss: 0.4624 - val_d_loss: 0.5627 - val_v_ccc: 0.4367 - val_a_ccc: 0.5422 - val_d_ccc: 0.4420
Epoch 14/50
6295/6295 [==============================] - 108s 17ms/step - loss: 0.5199 - v_loss: 0.1431 - a_loss: 0.1787 - d_loss: 0.1979 - v_ccc: 0.8568 - a_ccc: 0.8212 - d_ccc: 0.8020 - val_loss: 1.5893 - val_v_loss: 0.5349 - val_a_loss: 0.4789 - val_d_loss: 0.5884 - val_v_ccc: 0.4715 - val_a_ccc: 0.5246 - val_d_ccc: 0.4146
Epoch 15/50
6295/6295 [==============================] - 108s 17ms/step - loss: 0.4942 - v_loss: 0.1373 - a_loss: 0.1704 - d_loss: 0.1866 - v_ccc: 0.8626 - a_ccc: 0.8297 - d_ccc: 0.8135 - val_loss: 1.5623 - val_v_loss: 0.5386 - val_a_loss: 0.4683 - val_d_loss: 0.5702 - val_v_ccc: 0.4691 - val_a_ccc: 0.5355 - val_d_ccc: 0.4331
Epoch 16/50
6295/6295 [=====

6295/6295 [==============================] - 114s 18ms/step - loss: 1.0173 - v_loss: 0.3225 - a_loss: 0.3056 - d_loss: 0.3892 - v_ccc: 0.6776 - a_ccc: 0.6945 - d_ccc: 0.6106 - val_loss: 1.5510 - val_v_loss: 0.5937 - val_a_loss: 0.4415 - val_d_loss: 0.5263 - val_v_ccc: 0.4078 - val_a_ccc: 0.5625 - val_d_ccc: 0.4788
Epoch 4/50
6295/6295 [==============================] - 114s 18ms/step - loss: 0.9384 - v_loss: 0.2775 - a_loss: 0.2988 - d_loss: 0.3620 - v_ccc: 0.7225 - a_ccc: 0.7011 - d_ccc: 0.6380 - val_loss: 1.5499 - val_v_loss: 0.5824 - val_a_loss: 0.4334 - val_d_loss: 0.5429 - val_v_ccc: 0.4207 - val_a_ccc: 0.5678 - val_d_ccc: 0.4617
Epoch 5/50
6295/6295 [==============================] - 115s 18ms/step - loss: 0.8639 - v_loss: 0.2514 - a_loss: 0.2791 - d_loss: 0.3332 - v_ccc: 0.7485 - a_ccc: 0.7209 - d_ccc: 0.6667 - val_loss: 1.5502 - val_v_loss: 0.5954 - val_a_loss: 0.4322 - val_d_loss: 0.5347 - val_v_ccc: 0.4100 - val_a_ccc: 0.5685 - val_d_ccc: 0.4714
Epoch 6/50
6295/6295 [========

6295/6295 [==============================] - 108s 17ms/step - loss: 0.8046 - v_loss: 0.2313 - a_loss: 0.2658 - d_loss: 0.3076 - v_ccc: 0.7688 - a_ccc: 0.7342 - d_ccc: 0.6925 - val_loss: 1.5679 - val_v_loss: 0.5825 - val_a_loss: 0.4354 - val_d_loss: 0.5696 - val_v_ccc: 0.4249 - val_a_ccc: 0.5686 - val_d_ccc: 0.4386
Epoch 7/50
6295/6295 [==============================] - 108s 17ms/step - loss: 0.7628 - v_loss: 0.2126 - a_loss: 0.2594 - d_loss: 0.2909 - v_ccc: 0.7874 - a_ccc: 0.7405 - d_ccc: 0.7093 - val_loss: 1.5131 - val_v_loss: 0.5623 - val_a_loss: 0.4216 - val_d_loss: 0.5389 - val_v_ccc: 0.4443 - val_a_ccc: 0.5811 - val_d_ccc: 0.4616
Epoch 8/50
6295/6295 [==============================] - 108s 17ms/step - loss: 0.7054 - v_loss: 0.2011 - a_loss: 0.2364 - d_loss: 0.2678 - v_ccc: 0.7988 - a_ccc: 0.7636 - d_ccc: 0.7322 - val_loss: 1.5551 - val_v_loss: 0.5653 - val_a_loss: 0.4312 - val_d_loss: 0.5744 - val_v_ccc: 0.4421 - val_a_ccc: 0.5709 - val_d_ccc: 0.4319
Epoch 9/50
6295/6295 [========

...

6295/6295 [==============================] - 127s 20ms/step - loss: 0.9281 - v_loss: 0.2797 - a_loss: 0.2939 - d_loss: 0.3548 - v_ccc: 0.7203 - a_ccc: 0.7061 - d_ccc: 0.6455 - val_loss: 1.5868 - val_v_loss: 0.6035 - val_a_loss: 0.4330 - val_d_loss: 0.5663 - val_v_ccc: 0.4012 - val_a_ccc: 0.5706 - val_d_ccc: 0.4414
Epoch 5/50
6295/6295 [==============================] - 127s 20ms/step - loss: 0.8776 - v_loss: 0.2552 - a_loss: 0.2861 - d_loss: 0.3363 - v_ccc: 0.7447 - a_ccc: 0.7140 - d_ccc: 0.6637 - val_loss: 1.6594 - val_v_loss: 0.6731 - val_a_loss: 0.4641 - val_d_loss: 0.5415 - val_v_ccc: 0.3340 - val_a_ccc: 0.5401 - val_d_ccc: 0.4665
Epoch 6/50
6295/6295 [==============================] - 127s 20ms/step - loss: 0.8129 - v_loss: 0.2358 - a_loss: 0.2675 - d_loss: 0.3093 - v_ccc: 0.7641 - a_ccc: 0.7324 - d_ccc: 0.6907 - val_loss: 1.5593 - val_v_loss: 0.5956 - val_a_loss: 0.4282 - val_d_loss: 0.5475 - val_v_ccc: 0.4105 - val_a_ccc: 0.5758 - val_d_ccc: 0.4544
Epoch 7/50
6295/6295 [========

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



6295/6295 [==============================] - 148s 24ms/step - loss: 1.6597 - v_loss: 0.6599 - a_loss: 0.4554 - d_loss: 0.5443 - v_ccc: 0.3400 - a_ccc: 0.5446 - d_ccc: 0.4557 - val_loss: 2.2521 - val_v_loss: 0.7363 - val_a_loss: 0.8009 - val_d_loss: 0.7143 - val_v_ccc: 0.2695 - val_a_ccc: 0.1983 - val_d_ccc: 0.2801
Epoch 2/50
6295/6295 [==============================] - 127s 20ms/step - loss: 1.1274 - v_loss: 0.3730 - a_loss: 0.3288 - d_loss: 0.4256 - v_ccc: 0.6272 - a_ccc: 0.6712 - d_ccc: 0.5743 - val_loss: 1.6562 - val_v_loss: 0.6576 - val_a_loss: 0.4872 - val_d_loss: 0.5214 - val_v_ccc: 0.3486 - val_a_ccc: 0.5164 - val_d_ccc: 0.4789
Epoch 3/50
6295/6295 [==============================] - 126s 20ms/step - loss: 1.0045 - v_loss: 0.3143 - a_loss: 0.3098 - d_loss: 0.3807 - v_ccc: 0.6859 - a_ccc: 0.6901 - d_ccc: 0.6196 - val_loss: 1.6457 - val_v_loss: 0.6391 - val_a_loss: 0.4218 - val_d_loss: 0.5996 - val_v_ccc: 0.3674 - val_a_ccc: 0.5819 - val_d_ccc: 0.4050
Epoch 4/50
6295/6295 [========

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



2170/2170 [==============================] - 6s 3ms/step
[0.3817335069179535, 0.5545731782913208, 0.48453381657600403]
29
